<a href="https://colab.research.google.com/github/MChujyo/SolarFlarePrediction/blob/main/vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pip install torch torchvision transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from transformers import ViTFeatureExtractor, ViTForImageClassification, ViTConfig
from sklearn.model_selection import train_test_split

# データセットクラスの定義
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# データの準備

# ここでデータをNumPy配列等でロードする
images = np.load('/content/drive/MyDrive/2024/GeoSciAi/datas/train_mag.npy')
images = images.transpose(2,0,1)
labels = np.load('/content/drive/MyDrive/2024/GeoSciAi/datas/train_label.npy')
images = images.astype(np.float64)
labels = labels.astype(np.int16)

# データを訓練用とテスト用に分割
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# 前処理とデータローダーの設定
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # ViTの入力サイズに合わせる
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # 適切な正規化を行う
])

train_dataset = CustomDataset(train_images, train_labels, transform=transform)
test_dataset = CustomDataset(test_images, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

# ViTモデルのロード
config = ViTConfig.from_pretrained('google/vit-base-patch16-224', num_channels=1, num_labels=2)
model = ViTForImageClassification(config)


# トレーニングの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

# トレーニングループ
def train(epoch):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device).long()
        outputs = model(images).logits
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss: {loss.item()}")

# モデルのトレーニング
for epoch in range(10):
    train(epoch)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 0, Step 0, Loss: 0.5417093634605408
Epoch 0, Step 10, Loss: 0.4367561340332031
